<a href="https://colab.research.google.com/github/ShohamWeiss/NeuralNetworkTutorials/blob/master/TensorFlow_Tutorial_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save and Load Models

Install and Import

In [1]:
pip install -q pyyaml h5py # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.2.0


Example Dataset

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Define Model

In [5]:
def create_model():
  model = tf.keras.models.Sequential([
      keras.layers.Dense(512, activation='relu', input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model

model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Save Checkpoints during training

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weight
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

#Train model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) #Pass callback to training

Epoch 1/10
16/32 [==============>...............] - ETA: 0s - loss: 1.5214 - accuracy: 0.5605
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 1.1445 - accuracy: 0.6710 - val_loss: 0.6981 - val_accuracy: 0.7900
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4143 - accuracy: 0.8820
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4143 - accuracy: 0.8820 - val_loss: 0.5156 - val_accuracy: 0.8390
Epoch 3/10
31/32 [============================>.] - ETA: 0s - loss: 0.2706 - accuracy: 0.9325
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2705 - accuracy: 0.9320 - val_loss: 0.4435 - val_accuracy: 0.8610
Epoch 4/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2016 - accuracy: 0.9547
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [7]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


Creating new Untrained model

In [10]:
model = create_model()

#Evaluating
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3361 - accuracy: 0.0900
Untrained model, accuracy:  9.00%


Loading saved weights

In [13]:
#load weights
model.load_weights(checkpoint_path)

# Re-evaluate
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3869 - accuracy: 0.8710
Restored model, accuracy: 87.10%


Checkpoint callback options

In [14]:
#Include the epoch in the file name (use str.format)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#create a callback that saves weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

#create a new model
model = create_model()

#save the weights using the 'checkpoint_path' format
model.save_weights(checkpoint_path.format(epoch=0))

#train the model witht the callback
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [15]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [16]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [17]:
mode = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4704 - accuracy: 0.8760
restored model, accuracy: 87.60%


Manually save wieghts

In [21]:
#save
model.save_weights('./checkpoints/my_checkpoint')

#create a new model
model = create_model()

#restore the weights
model.load_weights('./checkpoints/my_checkpoint')

#evaluate
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}".format(100*acc))

32/32 - 0s - loss: 0.4704 - accuracy: 0.8760
Restored model, accuracy: 87.60


Save The Entire Model

In [22]:
# Create and train a new model
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel
!mkdir -p saved_model
model.save("saved_model/my_model")

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1771 - accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4368 - accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2992 - accuracy: 0.9120
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2143 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1547 - accuracy: 0.9730
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [24]:
!ls saved_model
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [25]:
new_model = tf.keras.models.load_model('saved_model/my_model')

#Check architechture
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4341 - accuracy: 0.8520
Restored model, accuracy: 85.20%
(1000, 10)


HDF5 Format

In [27]:
#Create and train new model
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save as HDF5 using the '.h5' extension
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1880 - accuracy: 0.6590
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4388 - accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2972 - accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2135 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1581 - accuracy: 0.9650


In [28]:
# Re-create model
new_model = tf.keras.models.load_model('my_model.h5')

# Show architecture
new_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [29]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4129 - accuracy: 0.8680
Restored model, accuracy: 86.80%
